In [2]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
from constants.glm_constants import *
from constants.behavioral_constants import *

import seaborn as sns

### Does time shifting actually work

In [3]:
session = "20180802"
behavior_path = SESS_BEHAVIOR_PATH.format(sess_name=session)
beh = pd.read_csv(behavior_path)

# filter trials 
valid_beh = behavioral_utils.get_valid_trials(beh)
feature_selections = behavioral_utils.get_selection_features(valid_beh)
valid_beh = pd.merge(valid_beh, feature_selections, on="TrialNumber", how="inner")
beh = behavioral_utils.get_feature_values_per_session(session, valid_beh)
beh = behavioral_utils.calc_feature_probs(beh)
beh = behavioral_utils.calc_feature_value_entropy(beh, num_bins=2, quantize_bins=True)

In [4]:
beh = behavioral_utils.shuffle_beh_by_shift(beh)

In [5]:
beh[:50]

,TrialNumber,BlockNumber,TrialAfterRuleChange,TaskInterrupt,ConditionNumber,Response,ItemChosen,TrialType,CurrentRule,LastRule,...,CYANProb,GREENProb,MAGENTAProb,YELLOWProb,ESCHERProb,POLKADOTProb,RIPPLEProb,SWIRLProb,FeatEntropy,FeatEntropyBin
0,591,2,0,NaN,641,Incorrect,3.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
1,592,2,1,NaN,627,Incorrect,3.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
2,593,2,2,NaN,808,Incorrect,3.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
3,594,2,3,NaN,783,Incorrect,1.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
4,595,2,4,NaN,1116,Incorrect,1.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
5,596,2,5,NaN,1113,Incorrect,1.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
6,597,2,6,NaN,755,Correct,0.0,8,CIRCLE,CYAN,...,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,2.484907,1
7,598,2,7,NaN,899,Correct,0.0,8,CIRCLE,CYAN,...,0.078664,0.078664,0.097342,0.078664,0.078664,0.097342,0.078664,0.078664,2.480357,1
8,599,2,8,NaN,940,Correct,0.0,8,CIRCLE,CYAN,...,0.076018,0.076018,0.093346,0.085905,0.076018,0.106303,0.076018,0.076018,2.475823,1
9,600,2,9,NaN,1111,Incorrect,3.0,8,CIRCLE,CYAN,...,0.073740,0.073740,0.089879,0.093539,0.082774,0.101875,0.073740,0.073740,2.472489,0


### Check next trial entropy

In [6]:
beh["NextTrialEntropyBin"] = beh["FeatEntropyBin"].shift(-1)


In [7]:
beh = beh[~beh["NextTrialEntropyBin"].isna()]
beh["NextTrialEntropyBin"] = beh["NextTrialEntropyBin"].astype(int)

In [14]:
bin_labels = pd.qcut(beh["FeatEntropy"], 10, labels=False, retbins=True)[1]

In [15]:
len(bin_labels)

11